In [ ]:
import pandas as pd

# Read in the data
baseline = pd.read_csv('allfeatures_norm_Baseline.csv')
# add a new column to display target variable
baseline['target'] = 1
followup = pd.read_csv('allfeatures_norm_Controls.csv')
# add a new column to display target variable
followup['target'] = 0

# concatenate the two dataframes
all_data = pd.concat([baseline, followup], ignore_index=True)

# subset data where label is 1
cistern_data = all_data[all_data['label'] == 1]

In [ ]:
!pip install optuna

In [ ]:
import numpy as np
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#warnings.simplefilter(action='ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix, classification_report

In [ ]:
# drop all columns with datatype object
cistern_data = cistern_data.select_dtypes(exclude=['object'])

In [ ]:
cistern_data.shape

In [ ]:
# scaler = MinMaxScaler()

# # split data into train and test

# X = cistern_data.drop(['label'], axis=1)
# y = cistern_data['target']

In [ ]:
# # feature selection using correlation

# # get correlation matrix
# corr_matrix = X.corr().abs()

# # get upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# # get columns with correlation greater than 0.95

# to_drop_ = [column for column in upper.columns if any(upper[column] > 0.95)]

# # # drop columns with correlation greater than 0.95
# # cistern_data = cistern_data.drop(cistern_data[to_drop], axis=1)

# # drop columns with correlation less than 0.1

# # to_drop = [column for column in upper.columns if any(upper[column] < 0.1)]

# to_drop = corr_matrix[corr_matrix['target'] < 0.1].index.tolist()
# # drop columns with correlation less than 0.1
# X = X.drop(X[to_drop+to_drop_], axis=1)

# X.shape

In [ ]:
# use lightgbm to select features

import lightgbm as lgb
from sklearn.model_selection import train_test_split

# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(cistern_data.drop('target', axis=1), cistern_data['target'], test_size=0.2, random_state=42,shuffle=True, stratify=cistern_data['target'])

# create lgb dataset
train_data = lgb.Dataset(X_train, label=y_train)

# set parameters
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'

# train the model
num_round = 100
bst = lgb.train(param, train_data, num_round)

# plot feature importance
import matplotlib.pyplot as plt
lgb.plot_importance(bst, figsize=(12, 6))
plt.show()

# select features with importance > 0.01
selected_features = []

for i in range(len(bst.feature_importance())):
    if bst.feature_importance()[i] > 0.01:
        selected_features.append(bst.feature_name()[i])

selected_features

In [ ]:
sorted(bst.feature_importance())[::-1]

In [ ]:
len(bst.feature_importance())

In [ ]:
len(selected_features)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Number of folds for cross-validation
n_folds = 5

colors = ['orange', 'green', 'blue', 'cyan', 'magenta']

# Set up K-Fold cross-validation
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

X = np.array(cistern_data[selected_features])
y = np.array(cistern_data['target'])

# Initialize lists to store results
accuracies = []
f1_scores = []
conf_matrices = []
roc = []
legend_label = []
fprl = []
tprl = []

plt.figure(figsize=(5, 5))
plt.title('ROC Curve')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([-0.1, 1.0])
plt.ylim([-0.1, 1.0])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

# Perform 5-fold cross-validation
fold = 0
for train_index, test_index in kf.split(X):
    # Split data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create and fit the model
    model = lgb.LGBMClassifier(learning_rate=0.01, n_estimators=1000, num_leaves=31, objective='binary', metric='accuracy ')
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate and store metrics
    accuracies.append(accuracy_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    conf_matrices.append(confusion_matrix(y_test, y_pred))
    roc.append(roc_auc_score(y_pred, y_test))

    feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_, selected_features)), columns=['Value', 'Feature'])

    # show top 20 features

    # plt.figure(figsize=(20, 10))
    # sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[:20])
    # plt.title('LightGBM Features (avg over folds)')
    # plt.tight_layout()
    # plt.show()

    # plot ROC curve
    probs = model.predict_proba(X_test)
    preds = probs[:,1]
    print(preds)
    print(y_test)
    fpr, tpr, thresholds = roc_curve(y_test, preds)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, colors[fold])
    # add legend for each fold with AUC score and color of line
    legend_label.append('Fold {} (AUC = {:.2f})'.format(fold, roc_auc))

    fold += 1

    print(legend_label)

plt.legend(['Baseline (AUC = 0.50)'] + legend_label, loc='lower right')
plt.show()

In [ ]:
# Calculate average metrics
avg_accuracy = np.mean(accuracies)
avg_f1_score = np.mean(f1_scores)

# Output average results
print("Average Accuracy: {:.2f}%".format(avg_accuracy * 100))
print("Average F1 Score: {:.2f}%".format(avg_f1_score * 100))

# Aggregate confusion matrices
total_conf_matrix = np.sum(conf_matrices, axis=0)

# Output the aggregated confusion matrix
print("Aggregated Confusion Matrix:")
print(total_conf_matrix)

# Output the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Average ROC
avg_roc = np.mean(roc)
print("Average ROC: {:.2f}%".format(avg_roc * 100))

# create heartmap of confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(total_conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Aggregated Confusion Matrix')
plt.show()


In [ ]:
# build the lightgbm model
clf = lgb.LGBMClassifier(learning_rate=0.01, n_estimators=1000, num_leaves=31, objective='binary', metric='accuracy', importance_type='gain')
clf.fit(cistern_data[selected_features], cistern_data['target'])

In [ ]:
# plot feature importance

feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_, selected_features)), columns=['Value', 'Feature'])

# show top 20 features

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[:20])
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

# plot ROC curve

In [ ]:
import shap
import joblib

xgb_exp = shap.TreeExplainer(clf, feature_perturbation='tree_path_dependent', feature_names=selected_features)
shap_values = xgb_exp.shap_values(cistern_data[selected_features])
shap.summary_plot(shap_values, cistern_data[selected_features], feature_names=selected_features)
'''shap.plots.beeswarm(shap_values = xgb_exp)'''

In [ ]:
# compute SHAP values
explainer = shap.Explainer(clf, cistern_data[selected_features])
shap_values = explainer(cistern_data[selected_features])

In [ ]:
shap.plots.bar(shap_values, max_display=15)


In [ ]:
# compute SHAP values
explainer = shap.Explainer(clf, cistern_data[selected_features])
shap_values = explainer(cistern_data[selected_features])

In [ ]:
shap.plots.violin(
    shap_values, features=cistern_data[selected_features], feature_names=selected_features, plot_type="layered_violin"
)